# Bert Classifier for First ICU Prediction

## Import & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

import numpy as np
np.set_printoptions(precision=2)

import pandas as pd
import pickle
import torch
from pathlib import Path

In [9]:
from bert_classifier.data_processor import read_df
from utils.splits import set_two_splits
from args import args
vars(args)

SyntaxError: invalid syntax (args.py, line 23)

In [4]:
path = Path('../data')

In [5]:
ori_df = pd.read_csv(path/'processed_dataset.csv')

In [6]:
df = set_two_splits(ori_df.copy(), 'val')
train_ex = read_df(df.loc[(df['split'] == 'train')], 'note', 'class_label')
val_ex = read_df(df.loc[(df['split'] == 'train')], 'note', 'class_label', set_type='val')